<h1>CURRENTS Credit Management Model</h1>


<h2>Model introduction</h2>

The model represents a single user who trades with other users (the 'ecosystem' at large) using a mutual credit currency ('Currents'). <br>A <b>credit management algorithm</b> comprises the key logic for managing the credit allowance.


### Questions

1. Given varying variables in the credit management logic, how is user behaviour effected?
2. Given different user types, what credit management logic best supports the overall aims of the ecosystem design (i.e. does their balance tend towards 'equilibrium').
3. Does the system accomodate different categories of users? For example, both high & low volume transactors? 

### Assumptions

1. Users who maintain equilibrium are rewarded with larger credit access. For some portion of users, their transaction volumes will increase over time as they become key influencers within their ecosystems.

### Constraints / Scope

* The intention of this model is to generate insights that wil inform the design of the 'Currents' credit management algorithm

## Visual System Mapping: Entity Relationship Diagram
<!-- https://en.wikipedia.org/wiki/Entity%E2%80%93relationship_model -->

## Visual System Mapping: Stock & Flow Diagram

## Mathematical Specification


### Credit Management:

Credit Allowance is determined by a combination of factors, including the user's 'integrity' and 'participation':

Conceptually:
- AVG_Balance (over time T) = Behaviour (good tends to zero)
- Volume (over time T) = Influence/participation (high = more influence)
- Standard Deviation = ?


#### Integrity Score
- Good = AVG_Balance tends to 0

Specified as a table:

<i>AVG Balance/Volume  | Integrity Score</i>
- 2.5% = 10
- 5% = 9
- 7.5% = 8
- etc.


#### Credit Allowance
- 'Credit Allowance' determined as a % of total volume over time period (T), multiplied by Integrity & Participation coefficients (e.g. 10% of 30-day volume)



# 0. Dependencies

In [2773]:
# Standard libraries: https://docs.python.org/3/library/
import math
from numpy import random

# Analysis and plotting modules
import pandas as pd
# import plotly
import plotly.express as px
from random import normalvariate

# cadCAD configuration modules
from cadCAD.configuration.utils import config_sim
from cadCAD.configuration import Experiment

# cadCAD simulation engine modules
from cadCAD.engine import ExecutionContext, Executor


# 1. State Variables & System Parameters

In [2774]:
transactionList = []
balanceHistory = [] # a record of the user's running balance

# states are defined as python dictionaries:
initial_state = {
    'userBalance' : 0,
    'avgUserBalance' : 0,
    'totalNumTransactions' : 0,
    'userCredit' : 250,
    'userCapacity' : 250,
    'sendVolume' : 0, # volume over X send transactions specified by 'creditCalcVolume' param
    'transaction' : 0
    
  #  'commMembers' : 1000,
  #  'commBalance' : 0,
  #  'commCredit' : initial_state['userCredit'] * initial_state['commMembers'],
  #  'commCapacity' : initial_state['commBalance'] + initial_state['commCredit']
}


# System parameters:
# (Timestep = 1 day)

system_params = {
    'integrity_coefficient': [0], # determines user's balance proclivity (0=balanced spending & receiving)
    'AVG_ammount': [100], # Average transaction ammount
    'creditCalcVolume': [30] # How many previous transactions to use in credit calculation
    
}

# system_params['userType'][0]['integrity_coefficient'][2]


MONTE_CARLO_RUNS = 10

seeds = [random.RandomState(i) for i in range(MONTE_CARLO_RUNS)] #flexible way to create unique seeds for each monte carlo run


# Policy functions

In [2775]:
# Generate random incoming and outgoing User transactions
def p_transact(params, substep, state_history, previous_state):
    
    run = previous_state['run']
    integ = params['integrity_coefficient']
    capacity = previous_state['userCapacity']
    
    # randomly switch between (-) and (+) transactions, indicating sending or receiving
    posNeg = [-1, 1]
    switch = posNeg[int(round(seeds[run-1].rand()))]
    
    # generate random transaction amount
    transaction = params['AVG_ammount'] * (seeds[run-1].rand()+0.5) 
    
    # randomly switch transaction to incoming/outgoing 
    transaction = int(round(transaction)) * switch
    
    
    
    y = seeds[run-1].rand()
    # Generate random large outgoing transaction (limited by available capacity)
    if(transaction < 0 and y >= 0.9999):
        if(transaction*20 > (-1*capacity)):
            transaction = transaction*20
        elif(transaction*10 > (-1*capacity)):
            transaction = transaction*10
        elif (transaction*5 > (-1*capacity)):
            transaction = transaction*5
        else: 
            transaction = (-1*capacity)*y
            

    
    # update total number of transactions
    x = previous_state['totalNumTransactions'] + 1
    
    
    if(previous_state['totalNumTransactions'] == 364):
        transactionList.clear()
        balanceHistory.clear()
    
    
    return {'transaction_ammount': transaction, 'totalNumTransactions': x}

    
# CHECKED
def p_updateUserCapacity(params, substep, state_history, previous_state):
    
    x = previous_state['userBalance'] + previous_state['userCredit']
    return {'newUserCapacity': x}


# CHECKED
# Sum X number of outgoing transactions based on 'creditCalcVolume' parameter (informs credit allowance)
def p_calculateSendVolume(params, substep, state_history, previous_state):
    
    x = 0
    posHistory = []
    y = params['creditCalcVolume']
    
    # append only outgoing transactions to new list
    for i in transactionList:
        if(i>0):
            posHistory.append(i)
            
    
    # filter last X transactions (defined by 'creditCalcVolume')
    filteredPosHistory = posHistory[(-1*y):]
    
    for p in filteredPosHistory:
        x = x + p
    
    
    return {'sendVolume': x}


def p_updateAVGBalance(params, substep, state_history, previous_state):
    
    x = 0
    y = params['creditCalcVolume']
    
    
    filBHist = balanceHistory[(-1*90):]
    #filBHist = balanceHistory # uncomment to calc AVG balance over entire period
    
    length = len(filBHist)
    
    # sum total
    for i in filBHist:
        x = x + i   
    
    # calc average
    if (length > 0):
        p = x / length
    else:
        p = 0
    
    if(len(balanceHistory) < 20):
        p = 0
    
    return {'avgUserBalance': p}


def p_updateUserCredit(params, substep, state_history, previous_state):
    
    # fetch appropriate variables
    avgBalance = previous_state['avgUserBalance']
    creditCalcVol = params['creditCalcVolume']
    sendVol = previous_state['sendVolume']
    x = previous_state['userCredit']
    initial = initial_state['userCredit']
    avgTrans = params['AVG_ammount']
    
    # convert negative balance to positive number to simplify if statements later
    # if (avgBalance < 0):
    #    avgBalance *= -1

    
    # Before we have enough behavioural data, stick with default credit limit
    
    if (len(transactionList) < creditCalcVol):
        x = initial
        
    else: # adjust credit limit (as some percentage of volume) based on AVG balance
    
    # LOGIC 1:
    #    if(avgBalance != 0):
    #        x = (sendVol / avgBalance)
    #    else:
    #        x = previous_state['userCredit']
    
    # LOGIC 2 (how to make continuous without exponential?)
    #    if(sendVol != 0):
    #        bv = avgBalance/sendVol
    #        if(bv<0.05):
    #            x = x*1.02
    #        elif(bv<0.1):
    #            x = x*1.01
    #        elif(bv<0.15):
    #            x = x*1
    #        elif(bv<0.2):
    #            x = x*0.95
    #        elif(bv<0.3):
    #            x = x*0.9
    #        elif(bv>=0.4):
    #            x = x*0.8

    # LOGIC 3 - Quadtratic relation (i.e. parabola)
        p = avgBalance / avgTrans
        
        if(sendVol != 0): # ensure no division by zero
            p = avgBalance / (sendVol)

        
        y = -500*(p**2) + (avgTrans/10) # quadratic 1
        
        x = x + y
        
        if(x<0): #if negative, reset to zero
            x = 0
        
    
    
    # activate line below to remove Credit Management:
    # x = previous_state['userCredit']
    
    return {'userCredit': x}
    
    

# State Update Functions

In [2776]:
# Update current balance based previous transaction
def s_userBalance(params, substep, state_history, previous_state, policy_input):
    
    newBalance = previous_state['userBalance'] + policy_input['transaction_ammount'] 
       
    # keep record of running balance over time (until we know how to access state_history, lol)
    balanceHistory.append(newBalance)
    
    return ('userBalance', newBalance)


# CHECKED
def s_totalNumTransactions(params, substep, state_history, previous_state, policy_input):
    x = policy_input['totalNumTransactions']
    return ('totalNumTransactions', x)

# CHECKED
def s_userCapacity(params, substep, state_history, previous_state, policy_input):
    x = policy_input['newUserCapacity']
    y = int(round(x))
    return ('userCapacity', y)

# CHECKED
def s_userSendVolume(params, substep, state_history, previous_state, policy_input):  
    x = policy_input['sendVolume']
    return ('sendVolume', x)

# CHECKED
def s_userCredit(params, substep, state_history, previous_state, policy_input):
    x =  policy_input['userCredit']
    y = int(round(x))
    return ('userCredit', y)

#CHECKED
def s_transaction(params, substep, state_history, previous_state, policy_input):
    
    x = policy_input['transaction_ammount']
    y = int(round(x))
    transactionList.append(y)
    
    return ('transaction', y)

# update current average balance (note: distinct from current 'userBalance')
def s_updateAVGBalance(params, substep, state_history, previous_state, policy_input):
    x = policy_input['avgUserBalance']
    return ('avgUserBalance', x)


In [2777]:
partial_state_update_blocks = [
    {
        'policies': {
            'transact': p_transact,
            'updateUserCapacity': p_updateUserCapacity,
            'calculateSendVolume': p_calculateSendVolume,
            'avgUserBalance' : p_updateAVGBalance,
            'updateUserCredit': p_updateUserCredit
        },
        'variables': {
            'userBalance': s_userBalance,
            'userCredit': s_userCredit,
            'userCapacity': s_userCapacity,
            'sendVolume': s_userSendVolume,
            'transaction': s_transaction,
            'avgUserBalance': s_updateAVGbalance,
            'totalNumTransactions': s_totalNumTransactions,
            'avgUserBalance': s_updateAVGBalance
            
        }
    }
]



In [2778]:
from cadCAD import configs
del configs[:] # Clear any prior configs

#reset global variables
transactionList.clear()
balanceHistory.clear()

sim_config = config_sim({
    'N': 5,
    'T': range(2000),
    'M': system_params
})


experiment = Experiment()
experiment.append_configs(
    initial_state = initial_state,
    partial_state_update_blocks = partial_state_update_blocks,
    sim_configs = sim_config
)



In [2779]:
exec_context = ExecutionContext()
run = Executor(exec_context=exec_context, configs=configs)

(system_events, tensor_field, sessions) = run.execute()


                  ___________    ____
  ________ __ ___/ / ____/   |  / __ \
 / ___/ __` / __  / /   / /| | / / / /
/ /__/ /_/ / /_/ / /___/ ___ |/ /_/ /
\___/\__,_/\__,_/\____/_/  |_/_____/
by cadCAD

Execution Mode: local_proc
Configuration Count: 1
Dimensions of the first simulation: (Timesteps, Params, Runs, Vars) = (2000, 3, 5, 7)
Execution Method: local_simulations
SimIDs   : [0, 0, 0, 0, 0]
SubsetIDs: [0, 0, 0, 0, 0]
Ns       : [0, 1, 2, 3, 4]
ExpIDs   : [0, 0, 0, 0, 0]
Execution Mode: parallelized
Total execution time: 1.13s


In [2780]:
simulation_result = pd.DataFrame(system_events)
simulation_result.head(50)

,userBalance,avgUserBalance,totalNumTransactions,userCredit,userCapacity,sendVolume,transaction,simulation,subset,run,substep,timestep
0,0.0,0.000000,0,250,250,0,0,0,0,1,0,0
1,122.0,0.000000,1,250,250,0,122,0,0,1,1,1
2,214.0,0.000000,2,250,372,122,92,0,0,1,1,2
3,75.0,0.000000,3,250,464,214,-139,0,0,1,1,3
4,-54.0,0.000000,4,250,325,214,-129,0,0,1,1,4
5,89.0,0.000000,5,250,196,214,143,0,0,1,1,5
6,37.0,0.000000,6,250,339,357,-52,0,0,1,1,6
7,174.0,0.000000,7,250,287,357,137,0,0,1,1,7
8,270.0,0.000000,8,250,424,494,96,0,0,1,1,8
9,156.0,0.000000,9,250,520,590,-114,0,0,1,1,9


### 8. Simulation Output Preparation

In [2781]:
# Get system events and attribute index
df = (pd.DataFrame(system_events)
        .assign(days=lambda df: df.timestep)
        .query('timestep > 400')
        .query('timestep < 1500')
     )

# Clean substeps
first_ind = (df.substep == 0) & (df.timestep == 0)
last_ind = df.substep == max(df.substep)
inds_to_drop = (first_ind | last_ind)
df = df.loc[inds_to_drop].drop(columns=['substep'])

# Attribute parameters to each row
df = df.assign(**configs[0].sim_config['M'])
for i, (_, n_df) in enumerate(df.groupby(['simulation', 'subset', 'run'])):
    df.loc[n_df.index] = n_df.assign(**configs[i].sim_config['M'])
    

In [2782]:
pd.options.plotting.backend = "plotly"

# plot something else by querying the dataframe
fig_df = df.query('run == 1')
# fig_df = df

fig_df.plot(
    kind='line',
    x='timestep',
    y=['avgUserBalance', 'userCredit', 'transaction']
)

In [2783]:
fig = px.line(
    df,
    x='timestep',
    y=['avgUserBalance', 'userCredit'],
    facet_row='simulation',
    facet_col='run',
    height=400,
    template='seaborn'
)

fig.update_layout(
    margin=dict(l=30, r=20, t=20, b=20),
)

fig.show()

In [2784]:
# fig_df = df.query()
fig_df = df

fig = px.scatter(
    fig_df,
    x=fig_df.days,
    y=fig_df.userBalance,
    color=fig_df.integrity_coefficient.astype(str),
    opacity=0.1,
    trendline="lowess",
    labels={'color': 'integrity_coefficient'}
)

fig.show()

In [2785]:
fig3 = px.scatter(
    df[df.timestep >= 30],
    x='avgUserBalance',
    y='userCredit',
    color='timestep'
)
fig3.show()